<a href="https://colab.research.google.com/github/orlgulendam/BSG/blob/main/BSGRS%C3%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import hashlib
import struct

# --- SABİTLER ---
ROUNDS = 16  # Feistel için standart tur sayısı
# Doğrusal Eşlik Üreteci Sabitleri (Rastgelelik için)
LKG_MULT = 1664525
LKG_INC = 1013904223

# --- YARDIMCI FONKSİYONLAR ---

def feistel_function(right_half, sub_key):
    """
    F Fonksiyonu: Sağ yarıyı ve anahtarı karıştırır.
    İşlem: (R XOR Key) * Sabit mod 2^32
    """
    temp = right_half ^ sub_key
    # 32 bit modüler çarpma (taşmaları atar)
    encrypted_val = (temp * LKG_MULT + LKG_INC) & 0xFFFFFFFF
    # Ekstra karıştırma için bit kaydırma
    return ((encrypted_val << 5) | (encrypted_val >> 27)) & 0xFFFFFFFF

def key_schedule(master_key_bytes):
    """
    128 bitlik anahtardan 16 adet 32 bitlik tur anahtarı üretir.
    """
    # Anahtarı 4 adet 32 bitlik tamsayıya çevir
    keys = list(struct.unpack('>4I', master_key_bytes))
    round_keys = []

    # Basit bir genişletme algoritması
    for i in range(ROUNDS):
        # Döngüsel olarak anahtar parçalarını kullan ve değiştir
        val = (keys[i % 4] + i * LKG_MULT) & 0xFFFFFFFF
        round_keys.append(val)

    return round_keys

# --- TEMEL FONKSİYONLAR ---

def Anahtar_Uret(parola):
    """Parolayı SHA-256 ile özetleyip ilk 16 byte'ı alır."""
    return hashlib.sha256(parola.encode()).digest()[:16]

def Sifrele(duz_metin, anahtar_bytes):
    # DÜZELTME BURADA: Gelen string verisini önce byte'a çeviriyoruz
    if isinstance(duz_metin, str):
        duz_metin_bytes = duz_metin.encode('utf-8')
    else:
        duz_metin_bytes = duz_metin

    # Padding (PKCS7) - Artık byte üzerine ekleme yapıyoruz
    pad_len = 8 - (len(duz_metin_bytes) % 8)
    duz_metin_bytes += bytes([pad_len] * pad_len)

    round_keys = key_schedule(anahtar_bytes)
    sifreli_veri = bytearray()

    # Blok blok işlem (8 byte = 64 bit)
    for i in range(0, len(duz_metin_bytes), 8):
        # Bloğu 2 adet 32 bitlik sayıya ayır (L, R)
        L, R = struct.unpack('>II', duz_metin_bytes[i:i+8])

        for tur in range(ROUNDS):
            temp = R
            # Feistel Formülü: L_yeni = R, R_yeni = L ^ F(R, k)
            f_out = feistel_function(R, round_keys[tur])
            R = L ^ f_out
            L = temp

        # Feistel ağlarında son turda swap yapılmaz (veya ters çevrilir)
        # Çıktıyı birleştir (L ve R yer değiştirmiş halde biter)
        sifreli_veri.extend(struct.pack('>II', R, L))

    return sifreli_veri.hex()

def Desifrele(sifreli_hex, anahtar_bytes):
    veri = bytes.fromhex(sifreli_hex)
    round_keys = key_schedule(anahtar_bytes)
    # Deşifreleme için anahtarları TERS çevir
    round_keys = round_keys[::-1]

    cozulmus_veri = bytearray()

    for i in range(0, len(veri), 8):
        # Şifreli veriyi al (Dikkat: Şifrelerken R, L diye bitirmiştik)
        R, L = struct.unpack('>II', veri[i:i+8])

        # Geriye doğru Feistel
        # Şifrelemenin tam tersi: Önce swap yapılır, sonra işlem
        for tur in range(ROUNDS):
            temp = L
            f_out = feistel_function(L, round_keys[tur])
            L = R ^ f_out
            R = temp

        # Sonucu birleştir (Orijinal L, R sırasına döner)
        cozulmus_veri.extend(struct.pack('>II', L, R))

    # Padding temizle
    pad_len = cozulmus_veri[-1]
    return cozulmus_veri[:-pad_len].decode('utf-8')

# --- TEST VE RAPORLAMA KISMI ---

def Raporu_Baslat():
    print("==================================================================")
    print("          IRON-FEISTEL ALGORİTMASI - PERFORMANS RAPORU           ")
    print("==================================================================\n")

    # SENARYO 1: DOĞRULUK
    print(">> SENARYO 1: Veri Bütünlüğü Testi")
    p1 = "GizliMesaj"
    k1 = Anahtar_Uret("SuperSecretKey")

    enc = Sifrele(p1, k1)
    dec = Desifrele(enc, k1)

    print(f"[*] Girdi: {p1}")
    print(f"[*] Şifreli (Hex): {enc}")
    print(f"[*] Çıktı: {dec}")
    print(f"--> SONUÇ: {'BAŞARILI ✅' if p1 == dec else 'HATALI ❌'}")

    # SENARYO 2: ÇIĞ ETKİSİ (Avalanche Effect)
    print("\n>> SENARYO 2: Çığ Etkisi (Avalanche Effect) Analizi")
    print("Amaç: Anahtardaki 1 bitlik değişimin şifreli metindeki etkisi.")

    # Orijinal Şifreleme
    test_metni = "TestBlock" # 8 byte'tan uzun
    enc1 = Sifrele(test_metni, k1)

    # Anahtarı 1 bit değiştir
    k1_array = bytearray(k1)
    k1_array[0] = k1_array[0] ^ 1 # İlk byte'ın son bitini tersle
    k2 = bytes(k1_array)

    enc2 = Sifrele(test_metni, k2)

    print(f"[*] Anahtar 1 ile: {enc1}")
    print(f"[*] Anahtar 2 ile: {enc2}")

    # Değişim Oranı Hesaplama
    v1 = int(enc1, 16)
    v2 = int(enc2, 16)
    diff = v1 ^ v2
    # Şifreli metin uzunluğu (hex karakter sayısı * 4 = bit sayısı)
    total_bits = len(enc1) * 4
    changed_bits = bin(diff).count('1')
    ratio = (changed_bits / total_bits) * 100

    print(f"--> Toplam Bit: {total_bits}")
    print(f"--> Değişen Bit: {changed_bits}")
    print(f"--> Değişim Oranı: %{ratio:.2f}")

    print("\n[GENEL DEĞERLENDİRME]")
    if ratio > 40:
        print("Mükemmel! Algoritma %40'ın üzerinde değişim sağladı.")
        print("Bu, Feistel yapısının veriyi güçlü şekilde yaydığını gösterir.")
    else:
        print("Uyarı: Yayılma oranı düşük.")
    print("==================================================================")

Raporu_Baslat()

          IRON-FEISTEL ALGORİTMASI - PERFORMANS RAPORU           

>> SENARYO 1: Veri Bütünlüğü Testi
[*] Girdi: GizliMesaj
[*] Şifreli (Hex): 8bfa386a9a5a9a81af19837f2461b2e2
[*] Çıktı: GizliMesaj
--> SONUÇ: BAŞARILI ✅

>> SENARYO 2: Çığ Etkisi (Avalanche Effect) Analizi
Amaç: Anahtardaki 1 bitlik değişimin şifreli metindeki etkisi.
[*] Anahtar 1 ile: 7d1c3f650e23d792079cd555fca749b1
[*] Anahtar 2 ile: 9650bfd6c2c5e3882c7e2d7079f22598
--> Toplam Bit: 128
--> Değişen Bit: 60
--> Değişim Oranı: %46.88

[GENEL DEĞERLENDİRME]
Mükemmel! Algoritma %40'ın üzerinde değişim sağladı.
Bu, Feistel yapısının veriyi güçlü şekilde yaydığını gösterir.
